In [ ]:
#!/usr/bin/env python

# region
import numpy as np
import pandas as pd
from scipy import interpolate
import hvplot.pandas
import panel as pn
import holoviews as hv
import pyam
hvplot.extension('plotly')

start_year = 1990
scenario = "pathway"

data_start_year = 1990
data_end_year = 2020
proj_end_year = 2050

scenario= 'pathway'

# endregion

In [ ]:
def step2curve(name, variable):

    # Load the 'Input Data' tab of TNC's 'Positive Disruption NCS Vectors' google spreadsheet
    name = (
        pd.read_csv("podi/data/afolu_max_extent_and_flux2.csv")
        .drop(columns=["Region", "Country"])
        .rename(
            columns={
                "iso": "region",
                "Model": "model",
                "Scenario": "scenario",
                "Unit": "unit",
            }
        )
        .replace("Pathway", scenario)
    )

    # Create a 'variable' column that concatenates the 'Subvector' and 'Metric' columns
    name["variable"] = name.apply(
        lambda x: "|".join([x["Subvector"], x["Metric"]]), axis=1
    )
    name.drop(columns=["Subvector", "Metric"], inplace=True)

    # Filter for rows that have 'variable' (either 'Max extent' or 'Avg mitigation potential flux')
    name = name[name["variable"].str.contains(variable)]

    # If Value 1 is 'NA', set to 0
    name["Value 1"] = np.where(name["Value 1"].isna(), 0, name["Value 1"])

    # If Value 2 is 'NA', set to Value 1
    name["Value 2"] = np.where(
        name["Value 2"].isna(), name["Value 1"], name["Value 2"]
    )

    # If Value 3 is 'NA', set to Value 2
    name["Value 3"] = np.where(
        name["Value 3"].isna(), name["Value 2"], name["Value 3"]
    )

    # If Duration 1 is 'NA' or longer than proj_end_year - data_end_year, set to proj_end_year - data_end_year
    name["Duration 1 (Years)"] = np.where(
        (
            (name["Duration 1 (Years)"].isna())
            | (name["Duration 1 (Years)"] > proj_end_year - data_end_year)
        ),
        proj_end_year - data_end_year,
        name["Duration 1 (Years)"],
    )

    # If Duration 2 is 'NA', set to Duration 1
    name["Duration 2 (Years)"] = np.where(
        (name["Duration 2 (Years)"].isna()),
        name["Duration 1 (Years)"],
        name["Duration 2 (Years)"],
    )

    # If Duration 3 is 'NA', set to Duration 2
    name["Duration 3 (Years)"] = np.where(
        name["Duration 3 (Years)"].isna(),
        name["Duration 2 (Years)"],
        name["Duration 3 (Years)"],
    )

    # Create dataframe with timeseries columns
    name = pd.DataFrame(
        index=[
            name["model"],
            name["scenario"],
            name["region"],
            name["variable"],
            name["unit"],
            name["Value 1"],
            name["Duration 1 (Years)"],
            name["Value 2"],
            name["Duration 2 (Years)"],
            name["Value 3"],
            name["Duration 3 (Years)"],
        ],
        columns=np.arange(data_start_year, proj_end_year + 1, 1),
        dtype=float,
    )

    # Define a function that places values in each timeseries for the durations specified, and interpolates
    def rep(x):
        x0 = x
        x0.loc[data_start_year] = x.name[5]
        x0.loc[data_start_year + x.name[6]] = x.name[7]
        x0.loc[data_start_year + x.name[6] + x.name[8]] = x.name[9]
        x0.interpolate(axis=0, limit_area="inside", inplace=True)
        x.update(x0)
        return x

    name.update(name.apply(rep, axis=1))

    # Drop 'Value' and 'Duration' columns now that the timeseries have been created
    name = name.droplevel(
        [
            "Value 1",
            "Duration 1 (Years)",
            "Value 2",
            "Duration 2 (Years)",
            "Value 3",
            "Duration 3 (Years)",
        ]
    ).fillna(0)

    return name



In [11]:
# Load the 'Historical Observations' tab of TNC's 'Positive Disruption NCS Vectors' google spreadsheet
afolu_historical = (
    pd.DataFrame(pd.read_csv("afolu_historical.csv"))
    .drop(columns=["Region", "Country"])
    .rename(
        columns={
            "iso": "region",
            "Model": "model",
            "Scenario": "scenario",
            "Unit": "unit",
        }
    )
    .replace("Pathway", scenario)
)

# Create a 'variable' column that concatenates the 'Subvector' and 'Metric' columns
afolu_historical["variable"] = afolu_historical.apply(
    lambda x: "|".join([x["Subvector"], x["Metric"]]), axis=1
)
afolu_historical.drop(columns=["Subvector", "Metric"], inplace=True)

# Set the index to IAMC format
afolu_historical = afolu_historical.set_index(pyam.IAMC_IDX)

afolu_historical.columns = afolu_historical.columns.astype(int)

# For rows with no historical data prior to data_start_year, set data_start_year to zero
afolu_historical.update(
    afolu_historical.loc[
        afolu_historical.loc[:, :data_start_year].isna().all(axis=1), :
    ]
    .loc[:, data_start_year]
    .fillna(0)
)

# Set first year to 0 to allow for interpolation
afolu_historical.update(afolu_historical.iloc[:, 0].fillna(0))

# For rows with no historical data prior to data_end_year, set data_end_year to 0
afolu_historical.update(
    afolu_historical.loc[
        afolu_historical.loc[:, data_start_year + 1 : data_end_year]
        .isna()
        .all(axis=1),
        :,
    ]
    .loc[:, data_end_year]
    .fillna(0)
)

# Interpolate to fill historical data gaps
afolu_historical.interpolate(axis=1, limit_area="inside", inplace=True)

# Plot

# region

df = afolu_historical.droplevel(["model", "scenario"])
df = pd.melt(
    df.reset_index(),
    id_vars=["region", "variable", "unit"],
    var_name="year",
    value_name="Adoption",
).dropna()

select_region = pn.widgets.Select(
    options=df.region.unique().tolist(), name="Region"
)

#for units in ['m3','Mha','Tg dm']:
df = df[df.unit == 'Mha'].drop(columns="unit")

@pn.depends(select_region)
def exp_plot(select_region, 'Mha'):
    return (
        df[df.region == select_region]
        .sort_values(by="year")
        .hvplot(x="year", y='Mha', by=["variable"])
    )

pn.Column(select_region, exp_plot)